In [1]:
data_dir = "/Volumes/Data/Research/Healthy Brain Network/ActiGraph/gt3x_data"
wt = "/Volumes/Jon.Clucas/weartime.csv"
subdirs = ["in process", "processed", "raw"]

In [2]:
import calendar
from datetime import datetime, timedelta
import os
import numpy as np
import pandas as pd
import zipfile

In [3]:
weartime = pd.read_csv(wt)


def add_weartime(df, weartime=weartime):
    """
    Function to add weartime columns to a DataFrame
    
    Parameters
    ----------
    df : DataFrame
    
    weartime : DataFrame
    
    Returns
    -------
    df : DataFrame
    """
    weartime = weartime[["Subject Name", "Wear Periods", "Non-Wear Periods", "Wear %", "Non-Wear %"]]
    df["merge column"] = df["filename"].apply(lambda x: str(x)[:-18])
    df = df.merge(
        weartime,
        left_on='merge column',
        right_on='Subject Name',
        how='left'
    ).copy()
    df.drop("merge column", axis=1, inplace=True)
    return(df)


def read_zips(data_dir, subdirs):
    """
    Function to read in all specified raw csvs.
    
    Parameters
    ----------
    data_dir : string
        parent directory
        
    subdirs : list of strings
        subdirectories
        
    Returns
    -------
    dfs : dictionary
        dictionary of filenames and RAW csv file exported from ActiLife
    """
    dfs = list()
    for subdir in subdirs:
        sd = os.path.join(data_dir, subdir)
        for file in os.listdir(sd):
            if file.endswith(".gt3x"):
                zp = os.path.join(sd, file)
                zdict = {}
                try:
                    with zipfile.ZipFile(zp) as z:
                        with z.open("info.txt", "r") as f:
                            zdict['filename'] = file
                            for line in f:
                                try:
                                    key, value = line.decode("UTF-8").split(":")
                                    zdict[key] = value[1:-2]
                                except:
                                    continue
                    dfs.append(zdict)
                except:
                    print(zp)
    return(dfs)


def ticks_to_datetime(ticks):
    """
    Function to convert .NET ticks to datetime
    
    Parameter
    ---------
    ticks : int
    
    Returns
    -------
    dt : datetime
    """
    return(datetime(1, 1, 1) + timedelta(microseconds = int(ticks)//10))

In [4]:
z = read_zips(data_dir, subdirs)

In [5]:
df = pd.DataFrame.from_dict(z)

In [6]:
for col in ["Start Date", "Stop Date", "Download Date", "Last Sample Time"]:
    df[col] = df[col].map(lambda x: ticks_to_datetime(int(x)))

In [7]:
df["Days Recorded"] = df["Last Sample Time"].subtract(
    df["Start Date"]
)
df["Date Given"] = df["Start Date"].apply(lambda x: x.date())

In [8]:
df["End Month"] = df["Last Sample Time"].map(
    lambda x: calendar.month_name[x.month]
)
df["End Month (numeric)"] = df["Last Sample Time"].map(
    lambda x: x.month
)

In [9]:
pd.pivot_table(
    df,
    values="Days Recorded",
    index=["End Month (numeric)", "End Month"],
    columns="Sample Rate",
    aggfunc="count"
)

,Sample Rate,30,60
End Month (numeric),End Month,,
7,July,1.0,NaN
8,August,1.0,15.0
9,September,10.0,29.0
10,October,10.0,12.0


In [10]:
subtable = df[[
    "filename",
    "Serial Number",
    "Subject Name",
    "Sample Rate",
    "End Month",
    "Days Recorded",
    "Date Given",
    "Start Date",
    "Last Sample Time"
]].sort_values(by=[
    "Sample Rate",
    "Start Date",
    "Last Sample Time"
])

In [11]:
subtable = subtable.reset_index(drop=True)

In [12]:
returned = pd.read_excel(
    "/Volumes/Data/Research/Healthy Brain Network/ActiGraph/Devices_Returned.xlsx"
)
returned["date_device_given"] = returned["date_device_given"].apply(
                                    lambda x: datetime(
                                        int(str(x)[:4]),
                                        int(str(x)[4:-2]),
                                        int(str(x)[-2:])
                                    )
                                )
returned["Rand_ID"] = returned["Rand_ID"].astype(str)
returned["device_nbr"] = returned["device_nbr"].apply(lambda x: str(x).upper())

In [13]:
have = returned.merge(
    subtable,
    how='left',
    left_on=[
        'Rand_ID',
        'device_nbr'
    ],
    right_on=[
        'Subject Name',
        'Serial Number'
    ]
)
missing = have.loc[have["filename"].isnull()].copy()
have = have.loc[~have["filename"].isnull()].copy()
have.date_device_given = have.date_device_given.apply(lambda x: x.date())

In [14]:
missing = returned.loc[returned["Rand_ID"].isin(missing["Rand_ID"])].copy()
missing.reset_index()
missing.date_device_given = missing.date_device_given.apply(lambda x: x.date())

In [15]:
missing = missing.merge(
    subtable,
    how='left',
    left_on=[
        'device_nbr',
        'date_device_given'
    ],
    right_on=[
        'Serial Number',
        'Date Given'
    ]
)
missing.sort_values("date_device_given", inplace=True)

In [16]:
have = pd.concat(
    [have, missing.loc[~missing["filename"].isnull()].copy()],
    ignore_index=True
).sort_values("Date Given").reset_index()

In [17]:
missing = missing.loc[missing["filename"].isnull()].copy()

In [18]:
extra_files = subtable.loc[~subtable["filename"].isin(have["filename"])].sort_values(
    "Start Date"
).reset_index(drop=True)

In [19]:
missing = returned.loc[returned["Rand_ID"].isin(missing["Rand_ID"])].copy()
missing.reset_index()
missing.date_device_given = missing.date_device_given.apply(lambda x: x.date())
missing = missing.merge(
    extra_files,
    how='left',
    left_on=[
        'device_nbr'
    ],
    right_on=[
        'Serial Number'
    ]
)

In [20]:
missing_different_dates = missing.loc[~missing["filename"].isnull()].copy()
missing_different_dates["date_dif"] = abs(
    missing_different_dates["Date Given"] -
    missing_different_dates["date_device_given"]
).astype(int)

mindif = {}
minind = {}
for x in set(missing_different_dates["Rand_ID"]):
    mindif[x] = min(missing_different_dates.loc[missing_different_dates["Rand_ID"] == x]["date_dif"].values)
    minind[x] = ((mindif[x] == missing_different_dates["date_dif"]) & (x == missing_different_dates["Rand_ID"]))
missing_different_dates = missing_different_dates.loc[
    sum(
        missing_different_dates["Rand_ID"].map(minind)
    ).apply(lambda x: bool(x))
].copy()
missing_different_dates.drop("date_dif", axis=1, inplace=True)

In [21]:
have = pd.concat(
    [have, missing_different_dates],
    ignore_index=True
).sort_values("Date Given").reset_index()
missing = missing.loc[missing["filename"].isnull()].copy().reset_index(drop=True)
extra_files = subtable.loc[~subtable["filename"].isin(have["filename"])].sort_values(
    "Start Date"
).reset_index(drop=True)

In [30]:
extra_files.loc[extra_files["Date Given"].isin(missing["date_device_given"])].sort_values("Date Given")

,filename,Serial Number,Subject Name,Sample Rate,End Month,Days Recorded,Date Given,Start Date,Last Sample Time
6,5397290 (2017-10-16).gt3x,MOS2D17170104,5397290,60,September,23 days 16:16:00,2017-08-23,2017-08-23 18:09:00,2017-09-16 10:25:00


In [33]:
extras = pd.DataFrame(columns = ["Rand_ID",
    "date_device_given",
    "device_nbr_as_entered",
    "device_nbr",
    "filename",
    "days recorded",
    "Sample Rate"])
extras[[
    "Rand_ID",
    "date_device_given",
    "device_nbr_as_entered",
    "device_nbr",
    "filename",
    "days recorded",
    "Sample Rate"
]] = extra_files.sort_values("Date Given")[[
    "Subject Name",
    "Date Given",
    "Serial Number",
    "Serial Number",
    "filename",
    "Days Recorded",
    "Sample Rate"]]
extras = add_weartime(extras)
extras.loc[extras["".to_csv(
    "/Volumes/Data/Research/Healthy Brain Network/ActiGraph/Extra_Files.csv",
    index=False
)

In [24]:
have_too_many = have.loc[have.duplicated(["Rand_ID", "date_device_given"], False)]

In [25]:
returned_kids = dict()
for kid in returned["Rand_ID"]:
    if kid in list(have["Rand_ID"]):
        returned_kids[kid] = (
            str(set(
                have_too_many.loc[have_too_many["Rand_ID"] == kid]["filename"].values
            )),
            "%.1f" % round(np.mean(
                have_too_many.loc[have_too_many["Rand_ID"] == kid]["Days Recorded"].values
                ) / np.timedelta64(1,'D'),
                1
            )
        ) if kid in list(have_too_many["Rand_ID"]) else ((
            have.loc[have["Rand_ID"] == kid]["filename"].values[0],
            "%.1f" % round(
                have.loc[have["Rand_ID"] == kid]["Days Recorded"].values[0] / np.timedelta64(1,'D'),
                1
            )
        ))
returned_kids = pd.DataFrame.from_dict(returned_kids, orient='index')
returned_kids.columns=["filename", "days recorded"]

In [26]:
returned = returned.merge(
    returned_kids, left_on="Rand_ID", right_index=True, how='outer'
).merge(
    have[["Rand_ID", "Sample Rate"]].drop_duplicates(), on="Rand_ID", how="left"
).sort_values(
    by=[
        "date_device_given",
        "device_nbr",
        "Rand_ID"
    ],
    ascending=True
).copy()
add_weartime(returned).to_csv(
    "/Volumes/Data/Research/Healthy Brain Network/ActiGraph/Devices_Returned.csv",
    index=False
)

In [27]:
missing

,Rand_ID,date_device_given,device_nbr_as_entered,device_nbr,filename,Serial Number,Subject Name,Sample Rate,End Month,Days Recorded,Date Given,Start Date,Last Sample Time
0,5853502,2017-07-26,MOS2D17170072,MOS2D17170072,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaT,NaT
1,5960577,2017-07-27,MOS2D17170066,MOS2D17170066,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaT,NaT
2,5793522,2017-08-23,MOS2D17170032,MOS2D17170032,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaT,NaT
3,5223025,2017-08-23,MOS2D17170034,MOS2D17170034,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaT,NaT
4,5068348,2017-09-13,MOS2d17170016,MOS2D17170016,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaT,NaT
